<a href="https://colab.research.google.com/github/Amin-Jalilzadeh/MY_RESUME/blob/main/V01%20Find%20Nearest%20Points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from scipy.spatial import cKDTree
from scipy import inf


In [3]:
from google.colab import files
 
 
uploaded = files.upload()

Saving bbbbd.csv to bbbbd.csv


In [ ]:
import pandas as pd
import io
 
df = pd.read_csv(io.BytesIO(uploaded['bbbbd.csv']))
print(df)

In [70]:
df['new_col'] = list(zip(df.X, df.Y))

In [71]:
print(df)

            X          Y      full_id      osm_id osm_type           amenity  \
0    4.355532  52.010006  n2599514291  2599514291     node          pharmacy   
1    4.360679  52.009304  n2599514104  2599514104     node         fast_food   
2    4.359838  52.011632  n2599515416  2599515416     node              cafe   
3    4.359775  52.012207  n2599515328  2599515328     node              cafe   
4    4.352679  52.012850  n2599515308  2599515308     node      kindergarten   
..        ...        ...          ...         ...      ...               ...   
512  4.358494  52.007798  n2599513703  2599513703     node  community_centre   
513  4.367810  52.002028  n9508339244  9508339244     node  community_centre   
514  4.363321  52.009686  n2599513554  2599513554     node         ice_cream   
515  4.359654  52.011546  n2599514605  2599514605     node              cafe   
516  4.363595  52.013296  n2599514567  2599514567     node         fast_food   

     emergency  defibrilla  defibril_1 

In [72]:
max_distance = 0.004 # Assuming lats and longs are in decimal degrees, this corresponds to 11.1 meters
points = [(4.3555324, 52.0100064), (4.3606789, 52.0093045), (4.4606789, 52.0093045), (4.3906789, 52.0193045)]
tree = cKDTree(points)


In [ ]:
type (points)

In [ ]:
tree

In [ ]:
(points)

In [ ]:
type (df.new_col)

In [ ]:
numbers_list = df.new_col.tolist()
print(type(numbers_list))

In [88]:
tree = cKDTree(numbers_list)


In [ ]:
numbers_list

In [76]:

# point_neighbors_list = [(4.358494, 52.0077984), (4.3678101, 52.0020277), (4.3633212, 52.0096864), (4.3596538, 52.0115459), (4.363595, 52.0132958)]
point_neighbors_list = [(4.377347, 52.01285)]

In [47]:
for point in points:
    distances, indices = tree.query(point, len(points), p=2, distance_upper_bound=max_distance)
    point_neighbors = []
    for index, distance in zip(indices, distances):
        if distance == inf:
            break
        point_neighbors.append(points[index])
    point_neighbors_list.append(point_neighbors)

In [90]:
for point in df.new_col:
    distances, indices = tree.query(point, len(df.new_col), p=2, distance_upper_bound=max_distance)
    point_neighbors = []
    for index, distance in zip(indices, distances):
        if distance == inf:
            break
        point_neighbors.append(df.new_col[index])
    point_neighbors_list.append(point_neighbors)

In [94]:
len(df.new_col)

517

In [36]:
point

(4.363595, 52.0132958)

In [91]:
len(point_neighbors_list)

1035

In [92]:
len(df)

517

In [ ]:
point_neighbors_list

In [62]:
import geopandas as gpd
import shapely

In [64]:

df # your pandas dataframe with 10k records
df_filt = point_neighbors_list# your filtered dataframe

# Create geometries from your lat-lons
geom_list = [shapely.geometry.Point(lon,lat) for lon,lat in zip(df["X"] ,df["Y"])] # check the ordering of lon/lat

# create geopandas geodataframe
gdf = gpd.GeoDataFrame(df, geometry=geom_list, crs={"init":"EPSG:4326"})

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
import geopandas as gpd
gdf = gpd.GeoDataFrame(
df, geometry=gpd.points_from_xy(df.long, df.lat), crs="EPSG:4326")
gdf=gdf.to_crs("EPSG:3857") #convert projection web mercator or find best epsg code, where your data have.
gdf['buffergeometry25miles'] = gdf['geometry'].buffer(40233.6) # 25 mile = 40233.6 meter radius
gdf.set_geometry('buffergeometry25miles',inplace=True)

givengdf = gpd.GeoDataFrame(
given_df, geometry=gpd.points_from_xy(given_df.long, given_df.lat), crs="EPSG:4326")
givengdf=givengdf.to_crs("EPSG:3857")

intersect_df = gpd.overlay(gdf, givengdf, how='intersection', keep_geom_type=False)

intersect_df is spatial intersection of two dataframe, you can make multi intersection with this method.